# markdown.obisidian.personal.reference
> Functions for managing references in an Obsidian.md math vault

In [ ]:
#| default_exp markdown.obsidian.personal.reference

In [ ]:
#| export
import glob
import os
from os import PathLike
from pathlib import Path
import re
import shutil
from typing import Union, Optional
import warnings

from trouver.helper import (
    path_name_no_ext, alphabet_to_alphabet_group
)
from trouver.markdown.markdown.file import (
    MarkdownFile, MarkdownLineEnum
)
from trouver.markdown.markdown.heading import (
    heading_title
)
from trouver.markdown.obsidian.links import (
    ObsidianLink, LinkType, links_from_text
)
from trouver.markdown.obsidian.personal.authors import find_author_file
from trouver.markdown.obsidian.personal.index_notes import ( 
    convert_title_to_folder_name
)
from trouver.markdown.obsidian.personal.notes import (
    notes_linked_in_note
)
from trouver.markdown.obsidian.personal.note_type import (
    type_of_note, PersonalNoteTypeEnum
)
from trouver.markdown.obsidian.vault import(
    VaultNote, all_note_paths_by_name, note_path_by_name,
    NoteDoesNotExistError, NoteNotUniqueError
)

# Getting a reference folder

In [ ]:
#| export
def index_note_for_reference(
        vault: PathLike, # The vault in which the reference folder resides.
        reference: Union[str, Path] # - The reference. Is either - a str, in which case the reference folder will be the folder containing the (unique) note of the name `_index_{reference}.md`, - or a `Path` object (not just a pathlike!) relative to `vault`, in which case the path will be the path to the reference folder. 
        ) -> VaultNote:
    """
    Returns the index note of the specified reference in the vault.
    
    The index note must be named `_index_{reference_name}.md`, where
    `{reference_name}` is the name of the reference and also the name
    of the reference folder.
    
    **Raises**

    - TypeError
        - If `reference` is not a str or PathLike.
    - NoteDoesNotExistError
        - If a note of the name `_index_{reference_name}.md` does not exist
        in the vault.
    - NoteNotUniqueError
        - If multiple notes of the name `_index_{reference_name}.md` are found
        in the vault. Even if multiple notes of the name exist, a
        NoteNotUniqueError may not be raised if those notes are not found.
        The current implementation does not actually raise this error.
    """
    if (not isinstance(reference, str)
            and not isinstance(reference, PathLike)):
        raise TypeError(
            "Expected `reference` to be a str or a PathLike, but got"
            f" {type(reference)} instead.")
    try:
        if isinstance(reference, str):
            reference_name = reference
            index_note = VaultNote(vault, name=f'_index_{reference_name}')
        elif isinstance(reference, PathLike):
            reference_name = Path(reference).name
            index_note = VaultNote(
                vault, rel_path=Path(reference) / f'_index_{reference_name}.md')
    except NoteDoesNotExistError as e:
        raise NoteDoesNotExistError(
            "Index note does not exist in the vault. It is expected to be named"
            f" `_index_{reference_name}.md`.")
    except NoteNotUniqueError as e:
        raise NoteNotUniqueError(
            "Index note is not unique in the vault. It is named"
            f" `_index_{reference_name}.md`.")
    return index_note



In [ ]:
#| export
def reference_directory(
        vault: PathLike, # The vault in which the reference folder resides.
        reference: Union[str, Path] # - The reference. Is either - a str, in which case the reference folder will be the folder containing the (unique) note of the name `_index_{reference}.md`, - or a `Path` object (not just a pathlike!) relative to `vault`, in which case the path will be the path to the reference folder. 
        ) -> Path:
    """
    Verifies that the specified reference exists, has a valid index note and
    return the relative path to the folder for the referece.
    
    Assumes that the reference folder has an index note named
    `_index_{reference_name}.md` and this note is the unique note in the vault
    with this filename.

    **Returns**
    - Path
        - Relative to `vault`.
    
    **Raises**
    - TypeError
        - If `reference` is not a str or PathLike.
    - NoteDoesNotExistError
        - If a note of the name `_index_{reference_name}.md` does not exist
        in the vault.
    - NoteNotUniqueError
        - If multiple notes of the name `_index_{reference_name}.md` are found
        in the vault.
    """
    return Path(index_note_for_reference(vault, reference).rel_path).parent